In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
import os
import numpy as np

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
train_datasets = "/kaggle/input/brain-tumor-classification-mri/Testing"
validation_datasets = "/kaggle/input/brain-tumor-classification-mri/Testing"

**Batch_size:** Number of training examples utilized in one iteration.A larger batch size often leads to faster training times, but it may require more memory.

**Image_size:** The dimensions of the input images. Here it is 224x224 pixels

**Epoch:** An epoch is one complete pass through the entire training dataset.Training for too few epochs may result in underfitting.

In [3]:
batch_size = 64
image_size = 224
epochs = 5

In [4]:
def prepare_the_datasets(train_datasets, validation_datasets, batch_size, image_size):

    train_datasets_generator = ImageDataGenerator(rescale=1./255,
                                                  shear_range = 0.2, 
                                                  zoom_range = 0.2, 
                                                  horizontal_flip = True, 
                                                  fill_mode = "nearest")


    validation_datasets_generator = ImageDataGenerator(rescale=1.0/255)


    train_datasets_generator_data = train_datasets_generator.flow_from_directory(
        batch_size = batch_size,
        directory = train_datasets,
        shuffle = True,
        target_size = (image_size, image_size),
        class_mode = "categorical"
    )

    validation_datasets_generator_data = validation_datasets_generator.flow_from_directory(
        batch_size = batch_size,
        directory = validation_datasets,
        shuffle = True,
        target_size = (image_size, image_size),
        class_mode = "categorical"
    )


    return train_datasets_generator_data, validation_datasets_generator_data

In [5]:
train_data , validation_data = prepare_the_datasets(train_datasets, validation_datasets, batch_size, image_size)

Found 394 images belonging to 4 classes.
Found 394 images belonging to 4 classes.


In [6]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense

# Load the pre-trained DenseNet121 model
densenet = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the pre-trained model
for layer in densenet.layers:
    layer.trainable = False

# Build your custom model for classification
model = densenet.output
model = GlobalAveragePooling2D()(model)
model = Dropout(rate=0.5)(model)
model = Dense(4, activation='softmax')(model)

# Create the final model
custom_model = tf.keras.models.Model(inputs=densenet.input, outputs=model)

# Compile the model (specify your optimizer, loss function, and metrics)
custom_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the summary of the model
custom_model.summary()

29084464/29084464 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 zero_padding2d (ZeroPaddin  (None, 230, 230, 3)          0         ['input_1[0][0]']             
 g2D)                                                                                             
                                                                                                  
 conv1/conv (Conv2D)         (None, 112, 112, 64)         9408      ['zero_padding2d[0][0]']      
                                                                                                  
 conv1/bn (BatchNormalizati  

In [7]:
model_checkpoint_filpath = "model_checkpoint.h5"
callbacks_checkpoints = ModelCheckpoint(

    filepath = model_checkpoint_filpath, 
    save_weights_only = True, 
    monitor = "val_accuracy",
    mode = "max", 
    save_best_only = True
)

In [8]:
history = custom_model.fit(train_data, 
                    steps_per_epoch = len(train_data),
                    epochs = epochs,
                    validation_data = validation_data, 
                    validation_steps = len(validation_data),
                    callbacks = [callbacks_checkpoints]
                    )

Epoch 1/5
7/7 [==============================] - 27s 1s/step - loss: 2.2345 - accuracy: 0.2335 - val_loss: 1.5400 - val_accuracy: 0.2741
Epoch 2/5
7/7 [==============================] - 7s 1s/step - loss: 1.8527 - accuracy: 0.3071 - val_loss: 1.4171 - val_accuracy: 0.3299
Epoch 3/5
7/7 [==============================] - 7s 1s/step - loss: 1.8772 - accuracy: 0.2995 - val_loss: 1.3233 - val_accuracy: 0.3782
Epoch 4/5
7/7 [==============================] - 7s 1s/step - loss: 1.7785 - accuracy: 0.3147 - val_loss: 1.2244 - val_accuracy: 0.4467
Epoch 5/5
7/7 [==============================] - 7s 1s/step - loss: 1.7054 - accuracy: 0.3376 - val_loss: 1.1466 - val_accuracy: 0.4949


In [12]:
loss, accuracy =  custom_model.evaluate(validation_data, batch_size = batch_size)

7/7 [==============================] - 2s 210ms/step - loss: 1.1466 - accuracy: 0.4949
